In [ ]:
%load_ext pycodestyle_magic

In [ ]:
import spacy
import pandas as pd
from collections import Counter

In [ ]:
# 1. 口コミデータの読み込み
df = pd.read_csv('text/usj.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

# 口コミに含まれている空行を削除
df['口コミ'] = df['口コミ'].replace('\n+', '\n', regex=True)

# 旅行の時期が欠損している口コミを削除する
df = df.dropna(subset=['旅行の時期'])

# 旅行の時期を年度に直し，「年度」という項目を追加
df['年度'] = df['旅行の時期'].apply(lambda x: x.year if x.month >= 4 else x.year-1)

In [ ]:
nlp = spacy.load("ja_ginza")

# 使用する単語の品詞とストップワードの指定
include_pos = ('NOUN', 'PROPN')
stopwords = ('こと', 'なし', 'ため', 'よう', 'ところ',
             'もの', 'あと', 'さん', '以上', '以外')

In [ ]:
# 2. 単語の頻度をカウントする関数を用意
def count_words(df):
    # 出現する単語をリストに追加
    words = []
    for doc in nlp.pipe(df["口コミ"]):
        words.extend([token.lemma_ for token in doc
                      if token.pos_ in include_pos and
                      token.lemma_ not in stopwords])

    # 単語の頻度をカウント
    counter = Counter(words)
    return counter


# 3. 年度ごとに口コミの単語の頻度をカウント
year2counter = {}
for y in range(2011, 2019):
    year2counter[y] = count_words(df[df['年度'] == y])

In [ ]:
#%%pycodestyle
# 4. まとめてカウントする単語を登録した辞書を用意
keywords = {'ハリーポッター': ['ハリーポッター', 'ハリポタ', 'ハリー'],
            'ミニオン': ['ミニオン', 'ミニオンズ', 'ミニオンパーク'],
            'フライングダイナソー': ['フライングダイナソー'],
            'ハロウィン': ['ハロウィン', 'ハロウィーン', 'ハローウィン',
                          'ハローイン', 'ハロイン'],
            'クリスマス': ['クリスマス', 'Xmas']}

In [ ]:
# 5. 年度ごとに単語の頻度を表示
records = []
for keyword, synonyms in keywords.items():
    record = {'キーワード': keyword}
    for y in range(2011, 2019):
        count = sum([year2counter[y][s] for s in synonyms])
        record[y] = count
    records.append(record)

df_res = pd.DataFrame(records)  # 単語の頻度を保存するデータフレーム
df_res = df_res.set_index('キーワード')
display(df_res)